# Train reward model with human feedback

The reward model is trained on a human-labeled dataset with the preferred `star_rating` for a given review.  The model flattens the human-labeled data from Ground Truth into (review, star_rating, ranking) tuples and provides a reward score for the RL-based model fine-tuning.

![Pipeline](img/generative_ai_pipeline_rlhf_plus.png)

![RLHF](img/rlhf_qa.png)

![Convert human ranking data into reward dataset](img/convert_groundtruth_ranking_data_to_reward_model_dataset_qa.png)

In [1]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=33242574848, available=31260741632, percent=6.0, used=1542275072, free=19587190784, active=2768596992, inactive=10001981440, buffers=1523712, cached=12111585280, shared=1466368, slab=467099648)


In [3]:
# %pip install --disable-pip-version-check -q \
#     transformers==4.26.1 \
#     datasets==2.9.0 \
#     accelerate==0.17.0 \
#     bitsandbytes==0.37.0 \
#     promptsource==0.2.3 \
#     trl==0.4.1 \
#     evaluate==0.4.0

In [4]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [5]:
import io
import json
import uuid
import time
import boto3
import botocore

# Amazon Python SDK clients
sagemaker = boto3.client("sagemaker", region)
a2i = boto3.client("sagemaker-a2i-runtime")
s3 = boto3.client("s3", region)

In [6]:
import os
import glob
import numpy as np
import argparse
import pprint
from collections import defaultdict

import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset, DataLoader

from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

# Retrieve the `human_loops_started`

In [7]:
%store -r human_loops_started

In [8]:
try:
    human_loops_started
except NameError:
    print("*** PLEASE RUN PREVIOUS NOTEBOOK BEFORE CONTINUING ***")

In [9]:
print(human_loops_started)

['5473f5ed-d880-4e4f-84a2-a8e0422d3adc', 'c2fc4085-d6ae-4204-bd89-c84145832388', 'e0f85945-5948-4632-906f-67c8a380d28a']


# Verify the Human Loops are Completed

In [10]:
import time

completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f"HumanLoop Name: {human_loop_name}")
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print("")
    while resp["HumanLoopStatus"] != "Completed":
        print(f"Waiting for HumanLoop to complete.")
        time.sleep(10)
        resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)
        print(f"Completed!")
        print("")

HumanLoop Name: 5473f5ed-d880-4e4f-84a2-a8e0422d3adc
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/ground-truth-star-rating-results/fd-ranking-e1d20f46-1c12-48ba-85eb-c5ba96856d06/2023/04/04/02/53/13/5473f5ed-d880-4e4f-84a2-a8e0422d3adc/output.json'}

Waiting for HumanLoop to complete.
Waiting for HumanLoop to complete.
Waiting for HumanLoop to complete.
Completed!

HumanLoop Name: c2fc4085-d6ae-4204-bd89-c84145832388
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/ground-truth-star-rating-results/fd-ranking-e1d20f46-1c12-48ba-85eb-c5ba96856d06/2023/04/04/02/53/13/c2fc4085-d6ae-4204-bd89-c84145832388/output.json'}

Waiting for HumanLoop to complete.
Completed!

HumanLoop Name: e0f85945-5948-4632-906f-67c8a380d28a
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/ground-truth-star-rating-resul

# View Human Labels  

Once the work is complete, Amazon GroundTruth stores the results in the specified S3 bucket and sends a Cloudwatch Event.  Here is a sample item labeled with GroundTruth in `jsonlines` format:
```
{
 "inputContent": {"taskObject": {
                         "prompt": "Who is Angela Merkel's favorite President of the United States?",
                         "responses": ["George Clinton", "Barack Obama"]}
                 },
 "humanAnswers": [{"answerContent": {
                        "ranking_1": "1", # ranking for 1st response (1 is High)
                        "ranking_2": "2"  # ranking for 2nd response (2 is Low)
                 }}]
}
```

# Prepare human-labeled data for RL/PPO training
Retrieve from GrountTruth and convert to a binary reward (-1, 1) for all rankings as follows:

From this:
```
prompt                                                              response           ranking

Who is Angela Merkel's favorite President of the United States?     George Clinton     1   # High
Who is Angela Merkel's favorite President of the United States?     Barack Obama       2   # Low
```

To this:
```
prompt                                                              response           reward

Who is Angela Merkel's favorite President of the United States?     George Clinton     0   # Low reward
Who is Angela Merkel's favorite President of the United States?     Barack Obama       1   # High reward
```

To this:
```
prompt                                                              response                               highest_ranked_response

Who is Angela Merkel's favorite President of the United States?     ["George Clinton", "Barack Obama"]     [0, 1]
```

# _Note:  If nothing is showing up below, you need to return to finish the previous notebook by labeling the data in Ground Truth!!_

In [17]:
import re
from pprint import pprint

human_feedback_items = []

for resp in completed_human_loops:
    human_feedback_s3_uri = resp["HumanLoopOutput"]["OutputS3Uri"]
    split_string = re.split("s3://" + bucket + "/", resp["HumanLoopOutput"]["OutputS3Uri"])
    key = split_string[1]
    
    response = s3.get_object(Bucket=bucket, Key=key)
    content = response["Body"].read().decode("utf-8")
    json_output = json.loads(content)
    print(json_output)

    prompt = json_output["inputContent"]['taskObject']['prompt']
    responses = json_output["inputContent"]['taskObject']['responses']
    response_1_ranking = json_output["humanAnswers"][0]["answerContent"]['response_1_ranking']
    response_2_ranking = json_output["humanAnswers"][0]["answerContent"]['response_2_ranking']
    
    human_feedback_item_1 = (prompt, responses[0], response_1_ranking)
    human_feedback_items.append(human_feedback_item_1)
    human_feedback_item_2 = (prompt, responses[1], response_2_ranking)
    human_feedback_items.append(human_feedback_item_2)

{'flowDefinitionArn': 'arn:aws:sagemaker:us-east-1:079002598131:flow-definition/fd-ranking-e1d20f46-1c12-48ba-85eb-c5ba96856d06', 'humanAnswers': [{'acceptanceTime': '2023-04-04T02:53:59.342Z', 'answerContent': {'response_1_ranking': '2', 'response_2_ranking': '1'}, 'submissionTime': '2023-04-04T02:54:04.779Z', 'timeSpentInSeconds': 5.437, 'workerId': 'e7232bf5ab67e176', 'workerMetadata': {'identityData': {'identityProviderType': 'Cognito', 'issuer': 'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_WEvHYVrSh', 'sub': '06e39925-66bd-45b1-b495-9081ad730e85'}}}], 'humanLoopName': '5473f5ed-d880-4e4f-84a2-a8e0422d3adc', 'inputContent': {'taskObject': {'prompt': 'Who was the President of the United States in 2010?', 'responses': ['Hilary Clinton', 'Barack Obama']}}}
{'flowDefinitionArn': 'arn:aws:sagemaker:us-east-1:079002598131:flow-definition/fd-ranking-e1d20f46-1c12-48ba-85eb-c5ba96856d06', 'humanAnswers': [{'acceptanceTime': '2023-04-04T02:54:10.380Z', 'answerContent': {'response_

In [18]:
df_human_feedback_items = pd.DataFrame(human_feedback_items, columns=['prompt', 'response', 'ranking'])
df_human_feedback_items.head(10)

,prompt,response,ranking
0,Who was the President of the United States in ...,Hilary Clinton,2
1,Who was the President of the United States in ...,Barack Obama,1
2,Who was the Chancellor of Germany in 2010?,Angela Merkel,1
3,Who was the Chancellor of Germany in 2010?,Meghan Markle,2
4,Who is Angela Merkel's favorite President of t...,George Clinton,2
5,Who is Angela Merkel's favorite President of t...,Barack Obama,1


# Convert ranking into 0 or 1 reward

In [19]:
num_rankings = 2
df_human_feedback_items['response'] = df_human_feedback_items['response'].apply(lambda response: str(response))
df_human_feedback_items['ranking'] = df_human_feedback_items['ranking'].apply(lambda ranking: str(abs(int(ranking) - num_rankings)))
df_human_feedback_items.head(10)

,prompt,response,ranking
0,Who was the President of the United States in ...,Hilary Clinton,0
1,Who was the President of the United States in ...,Barack Obama,1
2,Who was the Chancellor of Germany in 2010?,Angela Merkel,1
3,Who was the Chancellor of Germany in 2010?,Meghan Markle,0
4,Who is Angela Merkel's favorite President of t...,George Clinton,0
5,Who is Angela Merkel's favorite President of t...,Barack Obama,1


In [20]:
df_human_feedback_items_grouped_by_prompt = df_human_feedback_items.groupby('prompt', as_index=False).agg({'prompt' : 'first', 'response' : ','.join, 'ranking' : ','.join})
df_human_feedback_items_grouped_by_prompt

,prompt,response,ranking
0,Who is Angela Merkel's favorite President of t...,"George Clinton,Barack Obama","0,1"
1,Who was the Chancellor of Germany in 2010?,"Angela Merkel,Meghan Markle","1,0"
2,Who was the President of the United States in ...,"Hilary Clinton,Barack Obama","0,1"


In [21]:
df_human_feedback_items_grouped_by_prompt['response'] = df_human_feedback_items_grouped_by_prompt['response'].apply(lambda response: [s for s in response.split(',')])
df_human_feedback_items_grouped_by_prompt['ranking'] = df_human_feedback_items_grouped_by_prompt['ranking'].apply(lambda ranking: [int(s) for s in ranking.split(',')])
df_human_feedback_items_grouped_by_prompt

,prompt,response,ranking
0,Who is Angela Merkel's favorite President of t...,"[George Clinton, Barack Obama]","[0, 1]"
1,Who was the Chancellor of Germany in 2010?,"[Angela Merkel, Meghan Markle]","[1, 0]"
2,Who was the President of the United States in ...,"[Hilary Clinton, Barack Obama]","[0, 1]"


In [16]:
from datasets import Dataset

# Create Dataset objects (Arrow PyTables) from Pandas dataframes
human_feedback_dataset = Dataset.from_pandas(df_human_feedback_items_grouped_by_prompt)
human_feedback_dataset

Dataset({
    features: ['prompt', 'response', 'ranking'],
    num_rows: 3
})

# Train a reward model with human preference and alignment data
This is typically a language model initialized from the supervised-fine-tuned (SFT) model (trained in a previous notebook), but with an additional binary-classification layer placed on top.  This reward model is used to train the reinforcement-learning model in the next step.  The reinforcement-learning model is what is deployed into production to serve applications.

# TODO:  This should be bloomz (not bloom) at this point in the flow

In [ ]:
%store -r model_checkpoint

In [ ]:
model_checkpoint = 'bigscience/bloomz-560m'

In [ ]:
try:
    model_checkpoint
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous section before you continue.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(model_checkpoint)

In [ ]:
%store -r supervised_fine_tuned_model_path

In [ ]:
try:
    supervised_fine_tuned_model_path
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous section before you continue.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(supervised_fine_tuned_model_path)

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import evaluate
import numpy as np
import torch.nn as nn
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from transformers.utils import PaddingStrategy

In [ ]:
# Simulated (cooking show) output from SFT (Step 1) for Amazon Customer Review generative-classification task
#model_name = "bigscience/bloomz-560m" 

# Load the human comparisons dataset for tuning the reward model.
# Simulated output from HF Step 2 to be used to train reward for "helpfulness"
#ds = load_dataset("openai/summarize_from_feedback", name="comparisons") 

# Load the value-head model and tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# Turn the dataset into pairs of prompt + responses, where text_j is the preferred prompt + response and text_k is the other.
def turn_into_text_classification_format(examples):
    new_examples = {"text_j": [], "text_k": []}
    for prompt, response, ranking in zip(examples["prompt"], examples["response"], examples["ranking"]):
        # TODO:  Add a check to make sure there is only a single 0 and a single 1
        if len(response) != 2 or len(ranking) != 2 or ranking[0] not in (0, 1) or ranking[1] not in (0, 1):
            raise ValueError(
                f"There should be two responses with a ranking that is either 0 or 1. Received {len(response)} responses and {len(ranking)} rankings."
            )
            
        highest_ranked_response_index = ranking.index(1)

        new_examples["text_j"].append(
            #str(response[highest_ranked_response_index]) + " " + tokenizer.bos_token + " " + prompt
            prompt + " " + str(response[highest_ranked_response_index])
        )
        new_examples["text_k"].append(
            #str(response[0 if highest_ranked_response_index == 1 else 1]) + " " + tokenizer.bos_token + " " + prompt
            prompt + " " + str(response[0 if highest_ranked_response_index == 1 else 1])
        )

    return new_examples

# Tokenize the dataset.
def preprocess_function(examples):
    tokenized_j = tokenizer(examples["text_j"], truncation=True)
    tokenized_k = tokenizer(examples["text_k"], truncation=True)
    return {
        "input_ids_j": tokenized_j["input_ids"],
        "attention_mask_j": tokenized_j["attention_mask"],
        "input_ids_k": tokenized_k["input_ids"],
        "attention_mask_k": tokenized_k["attention_mask"],
    }


In [ ]:
num_proc = 8  # Can adjust to be higher if you have more processors. Should work even if you don't have 8 CPUs, though.
original_columns = human_feedback_dataset.column_names
print(original_columns)

human_feedback_binary_classification_dataset = human_feedback_dataset.map(turn_into_text_classification_format, batched=True, num_proc=num_proc, remove_columns=original_columns)

human_feedback_tokenized_dataset = human_feedback_binary_classification_dataset.map(preprocess_function, 
                                                                                    batched=True, 
                                                                                    num_proc=num_proc, 
                                                                                    remove_columns=["text_j", "text_k"])

print(human_feedback_tokenized_dataset)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=1)

# # Need to do this for gpt2, because it doesn't have an official pad token.
# tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
# Define the metric that we'll use for validation.
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, _ = eval_pred
    # Here, predictions is rewards_j and rewards_k.
    # We want to see how much of the time rewards_j > rewards_k.
    predictions = np.argmax(predictions, axis=0)
    labels = np.zeros(predictions.shape)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# We need to define a special data collator that batches the data in our j vs k format.
@dataclass
class RewardDataCollatorWithPadding:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        features_j = []
        features_k = []
        for feature in features:
            features_j.append({"input_ids": feature["input_ids_j"], "attention_mask": feature["attention_mask_j"]})
            features_k.append({"input_ids": feature["input_ids_k"], "attention_mask": feature["attention_mask_k"]})
        batch_j = self.tokenizer.pad(
            features_j,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch_k = self.tokenizer.pad(
            features_k,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch = {
            "input_ids_j": batch_j["input_ids"],
            "attention_mask_j": batch_j["attention_mask"],
            "input_ids_k": batch_k["input_ids"],
            "attention_mask_k": batch_k["attention_mask"],
            "return_loss": True,
        }
        return batch

In [ ]:
class RewardTrainer(Trainer):
    # Define how to compute the reward loss.
    def compute_loss(self, model, inputs, return_outputs=False):
        rewards_j = model(input_ids=inputs["input_ids_j"], attention_mask=inputs["attention_mask_j"])[0]
        rewards_k = model(input_ids=inputs["input_ids_k"], attention_mask=inputs["attention_mask_k"])[0]
        loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()
        if return_outputs:
            return loss, {"rewards_j": rewards_j, "rewards_k": rewards_k}
        return loss

# Define and parse arguments.
local_rank = 0
resume_from_checkpoint = False
deepspeed = None
per_device_train_batch_size = 16
per_device_eval_batch_size = 16
gradient_accumulation_steps = 4
learning_rate = 2e-5
weight_decay = 0.001
bf16 = False
num_train_epochs = 1

# Define the training args. Needs to be done before the model is loaded if you are using deepspeed.
training_args = TrainingArguments(
    output_dir=f"{model_checkpoint.replace('/', '_')}_reward_model",
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
#    evaluation_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=gradient_accumulation_steps,
#    deepspeed=deepspeed,
#    local_rank=local_rank,
    remove_unused_columns=False,
    label_names=[],
)
    
# Train the model, woohoo.
trainer = RewardTrainer(
    model=model,
    args=training_args,
    train_dataset=human_feedback_tokenized_dataset, #["train"],
#    eval_dataset=tokenized_ds["validation"],
    compute_metrics=compute_metrics,
    data_collator=RewardDataCollatorWithPadding(tokenizer=tokenizer),
)

trainer.train(resume_from_checkpoint)

In [ ]:
reward_model_path = './tmp_models/reward_model/'

trainer.save_model(reward_model_path)

In [ ]:
%store reward_model_path

In [ ]:
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_path, num_labels=1)

In [ ]:
from transformers import TextClassificationPipeline
from transformers import pipeline

#device = torch.device('cuda' if use_cuda else 'cpu')

pipeline_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

reward_model_pipeline = pipeline("text-classification", tokenizer=pipeline_tokenizer, model=reward_model)

In [ ]:
prompt = 'Given the following review:\nAwesome\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5'
response = 3
reward_model_pipeline.predict(prompt + ' ' + str(response))

In [ ]:
prompt = 'Given the following review:\nIf you are prepping for the end of the world this is one of those things that you should have installed on your-end-of-the-world-proof PC.  Hail to the great Yuri!\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5'
response = 4
reward_model_pipeline.predict(prompt + ' ' + str(response))

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>